In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

# Define the VAE model
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(15, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 15),
            nn.Sigmoid()
        )

        self.mu = nn.Linear(32, 32)
        self.logvar = nn.Linear(32, 32)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x):
        x = self.encoder(x)
        mu = self.mu(x)
        logvar = self.logvar(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar


# Define the loss function
# def loss_function(recon_x, x, mu, logvar):
#     BCE = nn.BCELoss(reduction='sum')
#     reconstruction_loss = BCE(recon_x, x.view(-1, 15))
#     KL_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
#     return reconstruction_loss + KL_divergence
def loss_function(recon_x, x, mu, logvar):
    MSE = nn.MSELoss(reduction="sum")
    reconstruction_loss = MSE(recon_x, x.view(-1, 15))
    KL_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return reconstruction_loss + KL_divergence

In [2]:
import pickle
import io
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == "torch.storage" and name == "_load_from_bytes":
            return lambda b: torch.load(io.BytesIO(b), map_location="cpu")
        else:
            return super().find_class(module, name)


with open('real_matrix_list.pickle', 'rb') as f:
    matrix_list = CPU_Unpickler(f).load()
# matrix_list = torch.load(io.BytesIO(b), map_location=torch.device("cpu"))

matrix_tensor = torch.stack(matrix_list)

In [3]:
# Prepare your dataset (replace this with your actual dataset)
# Example random data
data = matrix_tensor

# Normalize the data
data_min = data.min()
data_max = data.max()
data_normalized = (data - data_min) / (data_max - data_min)

# Initialize the model and optimizer
vae = VAE()
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

# Training loop
epochs = 100
for epoch in range(epochs):
    vae.train()
    train_loss = 0
    for matrix in data_normalized:
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(matrix.view(-1, 15))
        loss = loss_function(recon_batch, matrix, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch: {}, Loss: {:.4f}'.format(epoch, train_loss / len(data_normalized)))

Epoch: 0, Loss: 0.0845
Epoch: 1, Loss: 0.0533
Epoch: 2, Loss: 0.0518
Epoch: 3, Loss: 0.0504
Epoch: 4, Loss: 0.0487
Epoch: 5, Loss: 0.0492
Epoch: 6, Loss: 0.0487
Epoch: 7, Loss: 0.0478
Epoch: 8, Loss: 0.0479
Epoch: 9, Loss: 0.0472
Epoch: 10, Loss: 0.0470
Epoch: 11, Loss: 0.0465
Epoch: 12, Loss: 0.0468
Epoch: 13, Loss: 0.0468
Epoch: 14, Loss: 0.0463
Epoch: 15, Loss: 0.0462
Epoch: 16, Loss: 0.0462
Epoch: 17, Loss: 0.0464
Epoch: 18, Loss: 0.0461
Epoch: 19, Loss: 0.0462
Epoch: 20, Loss: 0.0459
Epoch: 21, Loss: 0.0460
Epoch: 22, Loss: 0.0460
Epoch: 23, Loss: 0.0459
Epoch: 24, Loss: 0.0460
Epoch: 25, Loss: 0.0458
Epoch: 26, Loss: 0.0459
Epoch: 27, Loss: 0.0459
Epoch: 28, Loss: 0.0458
Epoch: 29, Loss: 0.0459
Epoch: 30, Loss: 0.0456
Epoch: 31, Loss: 0.0457
Epoch: 32, Loss: 0.0458
Epoch: 33, Loss: 0.0456
Epoch: 34, Loss: 0.0457
Epoch: 35, Loss: 0.0456
Epoch: 36, Loss: 0.0455
Epoch: 37, Loss: 0.0454
Epoch: 38, Loss: 0.0456
Epoch: 39, Loss: 0.0454
Epoch: 40, Loss: 0.0455
Epoch: 41, Loss: 0.0454
Ep

In [5]:
print(vae)

VAE(
  (encoder): Sequential(
    (0): Linear(in_features=15, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=15, bias=True)
    (5): Sigmoid()
  )
  (mu): Linear(in_features=32, out_features=32, bias=True)
  (logvar): Linear(in_features=32, out_features=32, bias=True)
)


In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
# device = "cpu"

iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train, X_test, y_train, y_test = (
    torch.tensor(X_train, device=device, dtype=torch.float32),
    torch.tensor(X_test, device=device, dtype=torch.float32),
    torch.tensor(y_train, device=device, dtype=torch.long),
    torch.tensor(y_test, device=device, dtype=torch.long),
)

In [8]:
from sklearn.metrics import accuracy_score
from collections import namedtuple

class IrisClassifier(nn.Module):
    def __init__(self):
        super(IrisClassifier, self).__init__()
        self.linear = nn.Linear(4, 3)

    def forward(self, x):
        x = self.linear(x)
        return x

ModelInfo = namedtuple("ModelInfo", ["state_dict", "matrix", "accuracy"])

NUM_OF_MODELS = 500


# Generate weight matrices from the VAE model
generated_matrices = []
with torch.no_grad():
    vae.eval()
    for _ in range(NUM_OF_MODELS):
        z = torch.randn(1, 32).to(device)  # Assuming latent space dimension is 32
        generated_matrix = vae.decoder(z)
        generated_matrices.append(
            generated_matrix.view(3, -1)
        )  # Reshape to (3, 4+1) matrix

# Extract bias column from each weight matrix
generated_state_dicts = []
for matrix in generated_matrices:
    W = matrix[:, :-1]  # Extract weight matrix
    b = matrix[:, -1]  # Extract bias column
    state_dict = {"linear.weight": W, "linear.bias": b}
    generated_state_dicts.append(state_dict)

# Evaluate each generated model
evaluated_models = []
for state_dict in generated_state_dicts:
    model = IrisClassifier().to(device)
    model.load_state_dict(state_dict)
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        _, labels = torch.max(y_pred, 1)
        accuracy = accuracy_score(y_test.cpu().numpy(), labels.cpu().numpy())
    evaluated_models.append(
        ModelInfo(state_dict=state_dict, matrix=None, accuracy=accuracy)
    )

# Print evaluation results
for i, model_info in enumerate(evaluated_models):
    print(f"Model {i+1} - Accuracy: {model_info.accuracy}")

Model 1 - Accuracy: 0.98
Model 2 - Accuracy: 0.98
Model 3 - Accuracy: 0.98
Model 4 - Accuracy: 0.98
Model 5 - Accuracy: 0.98
Model 6 - Accuracy: 0.98
Model 7 - Accuracy: 0.98
Model 8 - Accuracy: 0.98
Model 9 - Accuracy: 0.98
Model 10 - Accuracy: 0.98
Model 11 - Accuracy: 0.98
Model 12 - Accuracy: 0.98
Model 13 - Accuracy: 0.98
Model 14 - Accuracy: 0.98
Model 15 - Accuracy: 0.98
Model 16 - Accuracy: 0.98
Model 17 - Accuracy: 0.98
Model 18 - Accuracy: 0.98
Model 19 - Accuracy: 0.98
Model 20 - Accuracy: 0.98
Model 21 - Accuracy: 0.98
Model 22 - Accuracy: 0.98
Model 23 - Accuracy: 0.98
Model 24 - Accuracy: 0.98
Model 25 - Accuracy: 0.98
Model 26 - Accuracy: 1.0
Model 27 - Accuracy: 0.98
Model 28 - Accuracy: 0.98
Model 29 - Accuracy: 0.98
Model 30 - Accuracy: 1.0
Model 31 - Accuracy: 0.98
Model 32 - Accuracy: 0.98
Model 33 - Accuracy: 0.98
Model 34 - Accuracy: 0.98
Model 35 - Accuracy: 0.98
Model 36 - Accuracy: 0.98
Model 37 - Accuracy: 0.98
Model 38 - Accuracy: 0.98
Model 39 - Accuracy: 0.

In [10]:
generated_matrices

[tensor([[0.6283, 0.8018, 0.2401, 0.1743, 0.7214],
         [0.5930, 0.5405, 0.5513, 0.4361, 0.6350],
         [0.4718, 0.3701, 0.7278, 0.8760, 0.3406]]),
 tensor([[0.6297, 0.8058, 0.2362, 0.1702, 0.7251],
         [0.5938, 0.5394, 0.5527, 0.4358, 0.6368],
         [0.4713, 0.3677, 0.7311, 0.8805, 0.3388]]),
 tensor([[0.6295, 0.8031, 0.2397, 0.1728, 0.7219],
         [0.5932, 0.5402, 0.5521, 0.4361, 0.6364],
         [0.4721, 0.3686, 0.7290, 0.8770, 0.3394]]),
 tensor([[0.6259, 0.7975, 0.2475, 0.1801, 0.7193],
         [0.5933, 0.5377, 0.5510, 0.4420, 0.6301],
         [0.4693, 0.3714, 0.7307, 0.8760, 0.3374]]),
 tensor([[0.6278, 0.8010, 0.2409, 0.1754, 0.7211],
         [0.5929, 0.5401, 0.5512, 0.4365, 0.6343],
         [0.4715, 0.3702, 0.7277, 0.8756, 0.3406]]),
 tensor([[0.6255, 0.7960, 0.2452, 0.1825, 0.7171],
         [0.5910, 0.5400, 0.5498, 0.4366, 0.6330],
         [0.4721, 0.3713, 0.7235, 0.8695, 0.3433]]),
 tensor([[0.6293, 0.8039, 0.2393, 0.1731, 0.7232],
         [0.5934, 0

In [ ]:

real_matrix_list = [real.matrix for real in real_models]
generated_matrix_list = [s.matrix for s in scrambled_models]

# Save the list to a file
with open("generated_models.pickle", "wb") as f:
    pickle.dump(ge, f)

with open("real_matrix_list.pickle", "wb") as f:
    pickle.dump(real_matrix_list, f)